# DistilBERT Modelini SQuAD Veri Seti ile Soru-Cevaplama (Question-Answering) İçin Fine-Tune Etme

Bu projede, **distilbert-base-uncased** modeli **SQuAD** veri seti ile **fine-tuning** işlemi uygulanmıştır.

Model, soru-cevaplama (question-answering) görevleri için **AutoTokenizer**, **AutoModelForQuestionAnswering**, **TrainingArguments** ve **Trainer** kullanılarak eğitilmiştir. Bu işlem, modelin **doğal dil işleme (NLP)** görevleri içinde, özellikle metinler üzerinden sorulara doğru yanıtlar verme yetisini artırmıştır.

### Kullanılan Donanım  

- Kaggle Notebook - GPU: T4 x2

![image](../imgs/qa.png)

In [4]:
!nvidia-smi

Sat Sep 28 20:09:53 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import warnings 
warnings.filterwarnings('ignore')

In [6]:
!pip install -q transformers datasets torch
!echo 'Installations Done!'

Installations Done!


In [7]:
import numpy as np 
import json 
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
from transformers.trainer_utils import EvalPrediction
from transformers import pipeline 

import torch

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [8]:
model_name = 'distilbert-base-uncased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 🤗 Hugging Face Datasets Kütüphanesi

### Parametreler

- **`path`**:
  - Yüklenecek veri setinin yolu veya ismi. Örnek: "imdb", "glue".

- **`name`**:
  - Yüklenecek veri setinin alt kümesi. Örnek: "sst2" (GLUE için).

- **`data_dir`**:
  - Veri dosyalarının bulunduğu dizin.

- **`data_files`**:
  - Yüklenecek veri dosyaları.

- **`split`**:
  - Veri setinin bölünmesi (örneğin "train", "test").

- **`cache_dir`**:
  - Verinin önbelleğe alınacağı dizin.

- **`features`**:
  - Özelliklerin açıkça belirtildiği yer.

- **`download_config`**:
  - İndirme yapılandırma ayarları.

- **`download_mode`**:
  - İndirme modu: "reuse_dataset_if_exists", "reuse_cache_if_exists", "force_redownload".

- **`verification_mode`**:
  - Veri setinin doğrulama modu.

- **`ignore_verifications`**:
  - Artık kullanılmıyor, doğrulamaları atlamak için.

- **`keep_in_memory`**:
  - Eğer True ise, veri seti bellekte tutulur.

- **`save_infos`**:
  - Eğer True ise, veri seti bilgileri kaydedilir.

- **`revision`**:
  - Yüklenecek veri setinin versiyonu veya commit ID'si.

- **`token`**:
  - Private veri setleri için kullanılır, bir token sağlar.

- **`use_auth_token`**:
  - Artık kullanılmıyor, oturum açma token'ı için.

- **`task`**:
  - Artık kullanılmıyor, veri seti yükleme görevini belirtmek için.

- **`streaming`**:
  - Eğer True ise, veri seti akış modunda yüklenir.

- **`num_proc`**:
  - Çok işlemcili veri işleme için işlemci sayısı.

- **`storage_options`**:
  - Depolama seçenekleri.

- **`trust_remote_code`**:
  - Eğer True ise, uzaktan kod çalıştırmaya izin verir.

- **`**config_kwargs`**:
  - Diğer ek yapılandırma argümanları.


In [9]:
def load_data():
    """
    ## Load the SQuAD dataset from the Hugging Face Hub. 

    Returns:
        - dataset: The SQuAD dataset.
    """
    return load_dataset('squad', split='train[:5%]', trust_remote_code=True), load_dataset('squad', split='validation[:2%]', trust_remote_code=True)

In [10]:
train_data, val_data = load_data()

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [11]:
print(train_data)
print('-')
print(json.dumps(train_data[0], indent=4))

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 4380
})
-
{
    "id": "5733be284776f41900661182",
    "title": "University_of_Notre_Dame",
    "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
    "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?",
    "answers"

## 🤗 Hugging Face Tokenizer Kütüphanesi

### Parametreler

- **`text`**:
  - Tokenize edilecek metin veya metinlerin listesi.

- **`text_pair`**:
  - İkinci bir metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`text_target`**:
  - Hedef metin veya metinlerin listesi (genellikle seq2seq modelleri için kullanılır).

- **`text_pair_target`**:
  - Hedef ikinci metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`add_special_tokens`**:
  - Özel tokenler ekler (örneğin [CLS], [SEP]). True olması, modeli daha iyi performans gösterir.

- **`padding`**:
  - Padding stratejisi: True, False, "longest", "max_length".

- **`truncation`**:
  - Kesme stratejisi: True, False, "longest_first", "only_first".

- **`max_length`**:
  - Maksimum token sayısı. Yüksek değerler daha fazla bilgi taşır ama daha fazla bellek kullanır.

- **`stride`**:
  - Kesme sırasında kayma boyutu. Uzun metinler için daha küçük değerler kullanışlı olabilir.

- **`is_split_into_words`**:
  - Eğer True ise, metin kelimelere bölünmüş olarak kabul edilir.

- **`pad_to_multiple_of`**:
  - Padding boyutunun bir katı olacak şekilde padding ekler.

- **`return_tensors`**:
  - Döndürülecek tensör tipi: 'pt', 'tf', 'np'.

- **`return_token_type_ids`**:
  - Eğer True ise, token tipi ID'lerini döndürür.

- **`return_attention_mask`**:
  - Eğer True ise, attention maskelerini döndürür.

- **`return_overflowing_tokens`**:
  - Eğer True ise, taşan tokenleri döndürür.

- **`return_special_tokens_mask`**:
  - Eğer True ise, özel token maskesini döndürür.

- **`return_offsets_mapping`**:
  - Eğer True ise, offset mapping döndürür.

- **`return_length`**:
  - Eğer True ise, token uzunluklarını döndürür.

- **`verbose`**:
  - Eğer True ise, işlem hakkında daha fazla bilgi verir.

- **`**kwargs`**:
  - Diğer ek argümanlar.


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## 🤗 Hugging Face AutoModelForQuestionAnswering

### Parametreler

- **`pretrained_model_name_or_path`**:
  - Modelin önceden eğitilmiş halinin adını veya yolunu belirtir. Aşağıdaki şekillerde kullanılabilir:
    - Hugging Face model deposundaki bir *model id*'si.
    - Model ağırlıklarını içeren bir klasörün yolu (örn., `./my_model_directory/`).
    - TensorFlow kontrol noktası dosyasına giden bir yol veya URL (örn., `./tf_model/model.ckpt.index`), bu durumda `from_tf` True olarak ayarlanmalıdır ve `config` argümanı verilmelidir.

- **`model_args`**:
  - Modele özel ek argümanlar, alttaki modelin `__init__()` metoduna aktarılır.

- **`config`**:
  - Modelin kullanılacak yapılandırma dosyası. Aşağıdaki durumlarda otomatik olarak yüklenebilir:
    - Model Hugging Face tarafından sağlanan bir modeldir ve *model id* stringi kullanılarak yüklenir.
    - Model [~PreTrainedModel.save_pretrained] kullanılarak kaydedilmişse ve verilen dizin üzerinden yeniden yüklenir.
    - Model yerel bir klasör üzerinden yüklenir ve klasörde *config.json* dosyası bulunur.

- **`state_dict`**:
  - Kayıtlı ağırlık dosyasından yüklenen bir state dictionary yerine kullanılacak bir state dictionary.

- **`cache_dir`**:
  - İndirilen model yapılandırmasının önbelleğe alınacağı dizin.

- **`from_tf`**:
  - Model ağırlıklarını bir TensorFlow kontrol noktası dosyasından yükler (bu seçenek True yapılmalıdır).

- **`force_download`**:
  - Model ağırlıklarının ve yapılandırma dosyalarının tekrar indirilip indirilmeyeceğini belirler (mevcut önbellek dosyaları üzerine yazılır).

- **`resume_download`**:
  - Artık kullanılmayan ve göz ardı edilen bir argümandır. İndirilenler mümkün olduğunda varsayılan olarak devam eder.

- **`proxies`**:
  - Her bir istek için kullanılacak proxy sunucularının sözlüğü. Örnek: `{'http': 'foo.bar:3128', 'https': 'foo.bar:4012'}`.

- **`output_loading_info`**:
  - Eksik anahtarlar, beklenmeyen anahtarlar ve hata mesajlarını içeren bir sözlük döndürülüp döndürülmeyeceğini belirtir.

- **`local_files_only`**:
  - Yalnızca yerel dosyaların kullanılıp kullanılmayacağını belirtir (örn., indirme işlemi yapılmaz).

- **`revision`**:
  - Kullanılacak model versiyonu. Bir dal adı, etiket adı veya commit id'si olabilir.

- **`trust_remote_code`**:
  - Hub'da tanımlanan özel modellerin yerel makinede çalıştırılmasına izin verilip verilmeyeceğini belirtir. Bu seçenek yalnızca güvenilen depolar için True yapılmalıdır.

- **`code_revision`**:
  - Model dışındaki kodlar için kullanılacak belirli revizyon. Bir dal adı, etiket adı veya commit id'si olabilir.

- **`kwargs`**:
  - Yapılandırma nesnesini güncellemek ve modeli başlatmak için kullanılacak diğer ek argümanlar.


In [13]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def preprocess_data(data):
    """
    ## Preprocess the data for training the model. 

    Args: 
        - data: The SQuAD dataset.
    
    Returns:
        - input_data: The preprocessed data that contains the input_ids, attention_mask, start_positions, and end_positions.
    """
    
    questions = data['question']
    contexts = data['context']
    
    input_data = tokenizer(questions, contexts, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
    
    start_positions = []
    end_positions = []
    for answer in data['answers']:
        start_positions.append(answer['answer_start'][0])
        end_positions.append(answer['answer_start'][0] + len(answer['text'][0]))
        
    
    input_data['start_positions'] = start_positions
    input_data['end_positions'] = end_positions
    
    return input_data

In [15]:
train_data = train_data.map(preprocess_data, batched=True)
val_data = val_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/211 [00:00<?, ? examples/s]

In [16]:
def compute_metrics(predictions: EvalPrediction):
    """
    ## Compute the metrics for the model.

    Args:
        - predictions: The predictions made by the model. It contains the predictions and the label_ids.
    
    Returns:
        - metrics: The metrics for the model such as accuracy, precision, recall, and f1 score.
    """
    start_token, end_token = predictions.predictions
    pred_start_positions, pred_end_positions = np.argmax(start_token, axis=1), np.argmax(end_token, axis=1)
    
    start_positions, end_positions = predictions.label_ids[0], predictions.label_ids[1]
    
    start_accuracy = accuracy_score(start_positions, pred_start_positions)
    end_accuracy = accuracy_score(end_positions, pred_end_positions)
    average_accuracy = (start_accuracy + end_accuracy) / 2
        
    start_precision_score = precision_score(start_positions, pred_start_positions, average='macro')
    end_precision_score = precision_score(end_positions, pred_end_positions, average='macro')
    average_precision_score = (start_precision_score + end_precision_score) / 2

    start_recall_score = recall_score(start_positions, pred_start_positions, average='macro')
    end_recall_score = recall_score(end_positions, pred_end_positions, average='macro')
    average_recall_score = (start_recall_score + end_recall_score) / 2

    start_f1_score = f1_score(start_positions, pred_start_positions, average='macro')
    end_f1_score = f1_score(end_positions, pred_end_positions, average='macro')
    average_f1_score = (start_f1_score + end_f1_score) / 2
    
    return {
        "accuracy" : average_accuracy * 100,
        'precision_score' : average_precision_score * 100,
        "recall_score" : average_recall_score * 100,
        "f1_score" : average_f1_score * 100
    }

## 🤗 Hugging Face TrainingArguments Kütüphanesi

### Parametreler

- **`output_dir`**:
  - Eğitim sırasında ve sonrasında sonuçların kaydedileceği dizin.

- **`overwrite_output_dir`**:
  - Eğer True ise, mevcut `output_dir` içeriğini üzerine yazar.

- **`do_train`**:
  - Eğitim işleminin yapılıp yapılmayacağını belirtir.

- **`do_eval`**:
  - Değerlendirme işleminin yapılıp yapılmayacağını belirtir.

- **`do_predict`**:
  - Tahmin işleminin yapılıp yapılmayacağını belirtir.

- **`eval_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`prediction_loss_only`**:
  - Eğer True ise, sadece kayıp hesaplanır, tahminler döndürülmez.

- **`per_device_train_batch_size`**:
  - Her bir cihaz (GPU/CPU) için eğitim batch boyutu.

- **`per_device_eval_batch_size`**:
  - Her bir cihaz (GPU/CPU) için değerlendirme batch boyutu.

- **`gradient_accumulation_steps`**:
  - Gradient biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama eğitim süresini uzatır.

- **`eval_accumulation_steps`**:
  - Değerlendirme biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama değerlendirme süresini uzatır.

- **`eval_delay`**:
  - Eğitim başladıktan sonra ilk değerlendirmenin yapılacağı adım sayısı.

- **`learning_rate`**:
  - Öğrenme oranı. Yüksek değerler daha hızlı öğrenir ama aşırı öğrenmeye (overfitting) yol açabilir.

- **`weight_decay`**:
  - Ağırlıkların küçülme oranı. Yüksek değerler modelin genelleme yeteneğini artırabilir ama öğrenme yavaşlar.

- **`adam_beta1`**:
  - Adam optimizasyon algoritması için beta1 parametresi.

- **`adam_beta2`**:
  - Adam optimizasyon algoritması için beta2 parametresi.

- **`adam_epsilon`**:
  - Adam optimizasyon algoritması için epsilon parametresi.

- **`max_grad_norm`**:
  - Gradientlerin maksimum normu. Büyük değerler modelin stabilitesini artırabilir.

- **`num_train_epochs`**:
  - Eğitim için epoch sayısı. Yüksek değerler daha fazla öğrenme sağlar ama aşırı öğrenmeye yol açabilir.

- **`max_steps`**:
  - Maksimum eğitim adım sayısı. -1 ise, tüm epoch'lar tamamlanır.

- **`lr_scheduler_type`**:
  - Öğrenme oranı zamanlayıcı tipi.

- **`lr_scheduler_kwargs`**:
  - Öğrenme oranı zamanlayıcısı için ek parametreler.

- **`warmup_ratio`**:
  - Öğrenme oranı ısınma oranı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`warmup_steps`**:
  - Öğrenme oranı ısınma adım sayısı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`log_level`**:
  - Log seviyesi. 'passive', 'info', 'warning', 'error' veya 'critical'.

- **`log_level_replica`**:
  - Çoklu GPU eğitiminde log seviyesi.

- **`log_on_each_node`**:
  - Çoklu node eğitiminde her node için loglama yapılır.

- **`logging_dir`**:
  - TensorBoard logları için dizin.

- **`logging_strategy`**:
  - Loglama stratejisi: 'no', 'steps' veya 'epoch'.

- **`logging_first_step`**:
  - Eğer True ise, ilk adımda loglama yapılır.

- **`logging_steps`**:
  - Kaç adımda bir loglama yapılacağı.

- **`logging_nan_inf_filter`**:
  - Eğer True ise, NaN ve sonsuz değerler loglanmaz.

- **`save_strategy`**:
  - Modelin kaydedilme stratejisi: 'no', 'steps' veya 'epoch'.

- **`save_steps`**:
  - Kaç adımda bir modelin kaydedileceği.

- **`save_total_limit`**:
  - Maksimum kaç model kaydedileceği.

- **`save_safetensors`**:
  - Modelin güvenli tensör formatında kaydedilip kaydedilmeyeceği.

- **`save_on_each_node`**:
  - Eğer True ise, her node'da model kaydedilir.

- **`save_only_model`**:
  - Eğer True ise, sadece model kaydedilir, optimizer ve lr scheduler kaydedilmez.

- **`restore_callback_states_from_checkpoint`**:
  - Callback durumlarının checkpoint'ten geri yüklenip yüklenmeyeceği.

- **`no_cuda`**:
  - Eğer True ise, GPU kullanılmaz.

- **`use_cpu`**:
  - Eğer True ise, CPU kullanılır.

- **`use_mps_device`**:
  - Eğer True ise, MacOS Metal Performance Shaders kullanılır.

- **`seed`**:
  - Rastgelelik için seed değeri.

- **`data_seed`**:
  - Veri yükleme için seed değeri.

- **`jit_mode_eval`**:
  - Eğer True ise, PyTorch JIT modu kullanılır.

- **`use_ipex`**:
  - Eğer True ise, Intel Extension for PyTorch kullanılır.

- **`bf16`**:
  - Eğer True ise, bfloat16 kullanılır.

- **`fp16`**:
  - Eğer True ise, float16 kullanılır.

- **`fp16_opt_level`**:
  - float16 optimizasyon seviyesi.

- **`half_precision_backend`**:
  - Yarı hassasiyet arka ucu: 'auto', 'amp' veya 'apex'.

- **`bf16_full_eval`**:
  - Eğer True ise, bfloat16 tam değerlendirme yapılır.

- **`fp16_full_eval`**:
  - Eğer True ise, float16 tam değerlendirme yapılır.

- **`tf32`**:
  - Eğer True ise, TensorFloat-32 kullanılır.

- **`local_rank`**:
  - Çoklu GPU eğitiminde lokal rank.

- **`ddp_backend`**:
  - DDP backend tipi: 'nccl', 'gloo', 'mpi'.

- **`tpu_num_cores`**:
  - TPU çekirdek sayısı.

- **`tpu_metrics_debug`**:
  - TPU metriklerinin debug bilgilerini içerir.

- **`debug`**:
  - Debug seçeneği: 'underflow_overflow', 'poor_optimization'.

- **`dataloader_drop_last`**:
  - Eğer True ise, dataloader son batch'i düşürür.

- **`eval_steps`**:
  - Değerlendirme adım sayısı.

- **`dataloader_num_workers`**:
  - Dataloader için çalışan sayısı. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`dataloader_prefetch_factor`**:
  - Dataloader için prefetch faktörü. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`past_index`**:
  - Geçmiş index.

- **`run_name`**:
  - Çalışma ismi.

- **`disable_tqdm`**:
  - Eğer True ise, tqdm progress bar devre dışı bırakılır.

- **`remove_unused_columns`**:
  - Eğer True ise, kullanılmayan sütunlar veri setinden kaldırılır.

- **`label_names`**:
  - Label isimleri.

- **`load_best_model_at_end`**:
  - Eğitim sonunda en iyi modelin yüklenip yüklenmeyeceği.

- **`metric_for_best_model`**:
  - En iyi modelin seçilmesi için kullanılacak metrik.

- **`greater_is_better`**:
  - Eğer True ise, metriklerde yüksek değerler daha iyi olarak kabul edilir.

- **`ignore_data_skip`**:
  - Eğer True ise, veri atlamaları göz ardı edilir.

- **`fsdp`**:
  - Fully Sharded Data Parallel ayarları.

- **`fsdp_min_num_params`**:
  - Fully Sharded Data Parallel için minimum parametre sayısı.

- **`fsdp_config`**:
  - Fully Sharded Data Parallel için ek ayarlar.

- **`fsdp_transformer_layer_cls_to_wrap`**:
  - Fully Sharded Data Parallel için transformer layer sınıfı.

- **`accelerator_config`**:
  - Accelerator için ek ayarlar.

- **`deepspeed`**:
  - DeepSpeed yapılandırması.

- **`label_smoothing_factor`**:
  - Label smoothing faktörü.

- **`optim`**:
  - Optimizasyon algoritması.

- **`optim_args`**:
  - Optimizasyon algoritması için ek argümanlar.

- **`adafactor`**:
  - Eğer True ise, Adafactor optimizasyon algoritması kullanılır.

- **`group_by_length`**:
  - Eğer True ise, input uzunluğuna göre gruplanır.

- **`length_column_name`**:
  - Uzunluk sütunu ismi.

- **`report_to`**:
  - Raporlama platformları (örneğin 'wandb').

- **`ddp_find_unused_parameters`**:
  - DDP'de kullanılmayan parametrelerin bulunması.

- **`ddp_bucket_cap_mb`**:
  - DDP bucket kapasitesi.

- **`ddp_broadcast_buffers`**:
  - DDP broadcast buffer'ları.

- **`dataloader_pin_memory`**:
  - Dataloader için bellek pinleme.

- **`dataloader_persistent_workers`**:
  - Dataloader için kalıcı çalışanlar.

- **`skip_memory_metrics`**:
  - Eğer True ise, bellek metrikleri atlanır.

- **`use_legacy_prediction_loop`**:
  - Eğer True ise, eski tahmin döngüsü kullanılır.

- **`push_to_hub`**:
  - Eğer True ise, model Hugging Face Hub'a itilir.

- **`resume_from_checkpoint`**:
  - Checkpoint'ten devam edilir.

- **`hub_model_id`**:
  - Hugging Face Hub model ID.

- **`hub_strategy`**:
  - Hub stratejisi.

- **`hub_token`**:
  - Hugging Face Hub token.

- **`hub_private_repo`**:
  - Eğer True ise, özel repo kullanılır.

- **`hub_always_push`**:
  - Eğer True ise, her zaman Hub'a itilir.

- **`gradient_checkpointing`**:
  - Eğer True ise, gradient checkpointing yapılır.

- **`gradient_checkpointing_kwargs`**:
  - Gradient checkpointing için ek argümanlar.

- **`include_inputs_for_metrics`**:
  - Eğer True ise, değerlendirme metrikleri için inputlar da dahil edilir.

- **`eval_do_concat_batches`**:
  - Eğer True ise, değerlendirme batch'leri birleştirilir.

- **`fp16_backend`**:
  - float16 arka ucu.

- **`evaluation_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`push_to_hub_model_id`**:
  - Hub model ID.

- **`push_to_hub_organization`**:
  - Hub organizasyon ID.

- **`push_to_hub_token`**:
  - Hub token.

- **`mp_parameters`**:
  - Model paralel parametreler.

- **`auto_find_batch_size`**:
  - Eğer True ise, batch boyutu otomatik bulunur.

- **`full_determinism`**:
  - Eğer True ise, deterministik eğitim yapılır.

- **`torchdynamo`**:
  - TorchDynamo kullanımı.

- **`ray_scope`**:
  - Ray scope.

- **`ddp_timeout`**:
  - DDP timeout süresi.

- **`torch_compile`**:
  - Eğer True ise, Torch compile kullanılır.

- **`torch_compile_backend`**:
  - Torch compile backend.

- **`torch_compile_mode`**:
  - Torch compile modu.

- **`dispatch_batches`**:
  - Batch dispatching.

- **`split_batches`**:
  - Batch splitting.

- **`include_tokens_per_second`**:
  - Eğer True ise, saniye başına token sayısı dahil edilir.

- **`include_num_input_tokens_seen`**:
  - Eğer True ise, görülen input token sayısı dahil edilir.

- **`neftune_noise_alpha`**:
  - Neptune noise alpha.

- **`optim_target_modules`**:
  - Optimizasyon hedef modüller.

- **`batch_eval_metrics`**:
  - Eğer True ise, batch değerlendirme metrikleri hesaplanır.

In [17]:
training_args = TrainingArguments(
    output_dir='./training-results',
    weight_decay=0.01, 
    num_train_epochs=10,
    report_to=[],
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy='epoch',
    learning_rate=1e-05,
    push_to_hub=False
)

## 🤗 Hugging Face Trainer Kütüphanesi

### Parametreler

- **`model`**:
  - Eğitim ve değerlendirme için kullanılacak model.

- **`args`**:
  - Eğitim argümanları, `TrainingArguments` sınıfı ile oluşturulur.

- **`data_collator`**:
  - Veriyi toplamak için kullanılan data collator.

- **`train_dataset`**:
  - Eğitim için kullanılacak veri seti.

- **`eval_dataset`**:
  - Değerlendirme için kullanılacak veri seti.

- **`tokenizer`**:
  - Metin verilerini modelin anlayabileceği sayısal değerlere dönüştürür.

- **`model_init`**:
  - Modelin başlatılması için kullanılacak fonksiyon.

- **`compute_metrics`**:
  - Değerlendirme metriklerini hesaplamak için kullanılacak fonksiyon.

- **`callbacks`**:
  - Eğitim sürecini izlemek ve kontrol etmek için kullanılacak callback listesi.

- **`optimizers`**:
  - Optimizasyon algoritması ve öğrenme oranı zamanlayıcısı.

- **`preprocess_logits_for_metrics`**:
  - Metrik hesaplama için logits'i ön işleme fonksiyonu.


In [18]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args, 
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

Step,Training Loss
500,6.155300
1000,5.954300


TrainOutput(global_step=1370, training_loss=5.991289169422902, metrics={'train_runtime': 1165.8306, 'train_samples_per_second': 37.57, 'train_steps_per_second': 1.175, 'total_flos': 5722605781401600.0, 'train_loss': 5.991289169422902, 'epoch': 10.0})

In [20]:
trainer.evaluate(val_data)

{'eval_loss': 6.2093305587768555,
 'eval_accuracy': 1.8957345971563981,
 'eval_precision_score': 0.30944311536416796,
 'eval_recall_score': 0.9580453659401028,
 'eval_f1_score': 0.34498880819780153,
 'eval_runtime': 2.1168,
 'eval_samples_per_second': 99.679,
 'eval_steps_per_second': 3.307,
 'epoch': 10.0}

In [21]:
model.save_pretrained('trained-model')
tokenizer.save_pretrained('trained-model-tokenizer')

('trained-model-tokenizer/tokenizer_config.json',
 'trained-model-tokenizer/special_tokens_map.json',
 'trained-model-tokenizer/vocab.txt',
 'trained-model-tokenizer/added_tokens.json',
 'trained-model-tokenizer/tokenizer.json')

In [22]:
qa_pipeline = pipeline('question-answering', model='trained-model', tokenizer='trained-model-tokenizer', device=device)

In [23]:
qa_pairs = [
    {
        "question": "When did the 2008 Sichuan earthquake occur?",
        "context": "The 2008 Sichuan earthquake, also known as the Wenchuan earthquake, occurred on May 12, 2008. It was a devastating 7.9 magnitude earthquake that struck the Sichuan province of China, resulting in widespread destruction and the loss of tens of thousands of lives.",
        "answer": "The 2008 Sichuan earthquake occurred on May 12, 2008."
    },
    {
        "question": "What are antibiotics used for?",
        "context": "Antibiotics are a type of medication used to treat bacterial infections. They work by killing or inhibiting the growth of bacteria. While antibiotics are powerful tools in fighting infection, their overuse can lead to antibiotic resistance, which is a growing concern in modern medicine.",
        "answer": "Antibiotics are used to treat bacterial infections."
    },
    {
        "question": "Who is Beyoncé?",
        "context": "Beyoncé is a world-renowned American singer, songwriter, actress, and producer. She gained fame as a member of Destiny's Child and later launched a successful solo career. Known for her powerful vocals, stage presence, and activism, Beyoncé has become one of the most influential entertainers of her generation.",
        "answer": "Beyoncé is a famous American singer, songwriter, and actress."
    },
    {
        "question": "Who was Frédéric Chopin?",
        "context": "Frédéric Chopin was a Polish composer and virtuoso pianist of the Romantic era, known for his solo piano compositions. His works, such as nocturnes, études, and waltzes, are celebrated for their emotional depth, technical mastery, and lyrical beauty.",
        "answer": "Frédéric Chopin was a Polish composer and pianist known for his Romantic era compositions."
    },
    {
        "question": "What is genocide?",
        "context": "Genocide is the deliberate and systematic destruction of a national, ethnic, racial, or religious group. It involves acts such as mass killings, causing serious harm, and forcibly transferring children, with the intent to annihilate the targeted group. The term was coined after World War II in response to the Holocaust.",
        "answer": "Genocide is the systematic destruction of a national, ethnic, racial, or religious group."
    },
    {
        "question": "What is an iPod?",
        "context": "The iPod is a portable media player developed by Apple Inc. It revolutionized the way people listened to music, offering a simple and stylish device that could store thousands of songs. First released in 2001, the iPod became one of the most iconic products of the digital music era.",
        "answer": "An iPod is a portable media player developed by Apple Inc."
    },
    {
        "question": "What is Montana known for?",
        "context": "Montana, a state in the northwestern United States, is known for its diverse landscapes, including mountains, plains, and rivers. It is home to Glacier National Park and part of Yellowstone National Park, offering vast opportunities for outdoor recreation. Montana is also referred to as 'Big Sky Country' for its expansive skies and scenic beauty.",
        "answer": "Montana is known for its diverse landscapes, national parks, and outdoor recreation."
    },
    {
        "question": "What is New York City famous for?",
        "context": "New York City is one of the most iconic cities in the world, known for its cultural diversity, bustling economy, and landmarks such as the Statue of Liberty, Times Square, and Central Park. As a global hub for finance, fashion, and entertainment, NYC is often called 'The City That Never Sleeps.'",
        "answer": "New York City is famous for its cultural diversity, landmarks, and being a global hub for finance and entertainment."
    },
    {
        "question": "What were Sino-Tibetan relations like during the Ming dynasty?",
        "context": "Sino-Tibetan relations during the Ming dynasty (1368–1644) were complex and marked by periods of both cooperation and conflict. The Ming court maintained a tributary relationship with Tibetan leaders, while also exerting influence through religious and political alliances. However, Tibet retained a significant degree of autonomy during this era.",
        "answer": "Sino-Tibetan relations during the Ming dynasty were marked by both cooperation and conflict, with Tibet maintaining some autonomy."
    },
    {
        "question": "What is 'Spectre' (2015 film) about?",
        "context": "'Spectre' is the 24th James Bond film, released in 2015, and stars Daniel Craig as Agent 007. The film follows Bond as he uncovers the secret organization known as SPECTRE, a global criminal syndicate with ties to his past. It combines espionage, action, and a personal journey for Bond, set against stunning international locations.",
        "answer": "'Spectre' is about James Bond uncovering the secret organization SPECTRE, which has ties to his past."
    },
    {
        "question": "What is 'The Legend of Zelda: Twilight Princess'?",
        "context": "'The Legend of Zelda: Twilight Princess' is an action-adventure video game developed by Nintendo. Released in 2006, the game follows the hero Link as he fights to save the kingdom of Hyrule from the encroaching Twilight Realm. It is praised for its immersive gameplay, engaging story, and darker tone compared to previous entries in the series.",
        "answer": "'The Legend of Zelda: Twilight Princess' is an action-adventure game where Link saves Hyrule from the Twilight Realm."
    },
    {
        "question": "What is the University of Notre Dame known for?",
        "context": "The University of Notre Dame, located in Indiana, USA, is renowned for its academic excellence, particularly in the fields of business, law, and engineering. It is also known for its rich athletic tradition, especially in college football, and its iconic campus landmarks such as the Golden Dome and Notre Dame Stadium.",
        "answer": "The University of Notre Dame is known for its academic excellence and rich athletic tradition, especially in college football."
    }
]

In [24]:
for data in qa_pairs:
    result = qa_pipeline(question=data['question'], context=data['context'])
    print(f"Question: {data['question']}")
    print(f"Answer: {result['answer']}")
    print('-')

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: When did the 2008 Sichuan earthquake occur?
Answer: Sichuan earthquake, also known as the Wenchuan earthquake,
-
Question: What are antibiotics used for?
Answer: their overuse can lead to antibiotic resistance,
-
Question: Who is Beyoncé?
Answer: as
-
Question: Who was Frédéric Chopin?
Answer: and virtuoso pianist of the Romantic era, known for his
-
Question: What is genocide?
Answer: with
-
Question: What is an iPod?
Answer: offering a simple and stylish device that
-
Question: What is Montana known for?
Answer: to as 'Big Sky Country' for its expansive skies and scenic beauty.
-
Question: What is New York City famous for?
Answer: and landmarks such as the Statue of Liberty, Times Square,
-
Question: What were Sino-Tibetan relations like during the Ming dynasty?
Answer: . However, Tibet
-
Question: What is 'Spectre' (2015 film) about?
Answer: film follows Bond as he uncovers the secret organization known
-
Question: What is 'The Legend of Zelda: Twilight Princess'?
Answer: 